# Human Activity Recognition using Inertial sensors and Neural Networks

Elia Bonetto, Filippo Rigotto. 

Deptartment of Information Engineering, University of Padova, Italy.

Human Data Analytics, a.y. 2018/2019

In [0]:
!nvidia-smi

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive/')
!ls /content/drive/My\ Drive/hda-project
os.chdir("/content/drive/My Drive/hda-project")

In [0]:
import numpy as np
import scipy as sp
import scipy.io
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import Image, SVG, clear_output
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16,10)
mpl.rcParams['axes.grid'] = True

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv1D, MaxPooling1D, Dropout
#from tensorflow.keras.layers import Conv2D, ZeroPadding2D, AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
#from tensorflow.keras.preprocessing import image

import ars_ds

## Data preprocessing

In [0]:
ds = ars_ds.ARSDataset('dataset/ARS_DLR_DataSet_V2.mat')
imu,labels = ds.load_dataset()
ds.save_dataset('dataset/ARSv2.npz')
num = imu.shape[0]

In [0]:
np.random.seed(1)
extraction = np.random.rand(num)
X_train = imu[extraction <= 0.8]
X_test  = imu[extraction > 0.8]
Y_train = labels[extraction <= 0.8]
Y_test  = labels[extraction > 0.8]

print("IMU shape:     " + str(imu.shape))
print("Labels shape:  " + str(labels.shape))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape:  " + str(X_test.shape))
print("Y_test shape:  " + str(Y_test.shape))

## Models

In [0]:
def conv1model(input_shape):
    # TODO
    
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(shape=input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    # X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> Batch Normalization -> ReLU Block applied to X
    X = Conv1D(32, 5, strides=1, name='conv0')(X_input)
    X = BatchNormalization(axis=2, name='bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling1D(2, name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    model = Model(inputs = X_input, outputs = X, name='Conv1Model')
    return model

## Training and evaluation

In [0]:
model = conv1model((X_train.shape[0],10))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=X_train, y=Y_train, epochs=100, batch_size=8)
model.save('output/conv1model.h5')
preds = model.evaluate(x=X_test, y=Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]) + " = " + str(preds[1]*100))

# Plot loss
plt.figure()
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Val loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Plot accuracy
plt.figure()
plt.plot(history.history['acc'], label='Train loss')
plt.plot(history.history['val_acc'], label='Val loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')